# Lagged Time-Series Clustering Simulations

In [1]:
import random
import os
from multiprocessing import Pool
from functools import partial
import numpy as np
from fastdtw import fastdtw
from sklearn.metrics import adjusted_rand_score
from ananke._ddtw import DDTW
from ananke._database_rework import TimeSeriesData
from ananke._ts_simulation import gen_table
from DBloomSCAN import BloomDistance

/home/mwhall/anaconda3/envs/ananke/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/mwhall/anaconda3/envs/ananke/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


Simulate data according to the above parameters, and insert it into an HDF5 file.

In [2]:
def generate_data(timepoints, nclust, nts_per_clust, nsr, shift_amount):
    if os.path.isfile("simulation.h5"):
        os.remove("simulation.h5")
    tsdata = TimeSeriesData("simulation.h5")
    dataset_names = tsdata.initialize_by_shape(timepoints=timepoints)
    nnoise = int(nsr*nclust*nts_per_clust)
    sim = gen_table(fl_sig=0, w_sig=6,
                    fl_bg=-6, w_bg=6,
                    bg_disp_mu=0, bg_disp_sigma=1,
                    sig_disp_mu2=0, sig_disp_sigma2=signal_variance,
                    n_clust=nclust, n_sig=nts_per_clust, n_tax_sig=1, n_bg=nnoise,
                    len_arima=2*nsamples, len_ts=nsamples, len_signal=nsamples-shift_amount)
    X = sim['table']
    Y = sim['signals']

    tsdata.register_timeseries(["ts%d" % (x,) for x in range(nclust*nts_per_clust+nnoise)])
    i = 0
    for ts in X:
        tsdata.set_timeseries_data(data=X[i,:], index=i, action='replace')
        i += 1
    del tsdata
    return Y

Cluster the data using a given distance function. Can do this in memory, or pull the data from disk. Only necessary for really large sets or my laptop.

In [3]:
def compute_distances(distance_function, in_memory=True):
    print("Loading HDF5 file")
    tsd = TimeSeriesData("simulation.h5")
    n_objects = tsd._h5t["data/timeseries/matrix"].shape[0]
    time_points = [int(x) for x in tsd._h5t["data/timeseries/time"][:]]
    time_delta = np.array(time_points[1:]) - np.array(time_points[0:-1])
    if in_memory:
        data_matrix = np.empty(tsd._h5t["data/timeseries/matrix"].shape)
        print("Loading data matrix into memory")
        tsd._h5t["data/timeseries/matrix"].read_direct(data_matrix)
        def retrieve_data(index):
            #If the data is too large for RAM, this can be swapped around to read from disk
            #data = tsd._h5t["data/timeseries/matrix"][index, :]
            #return return data/sum(data)
            return data_matrix[index,:]/sum(data_matrix[index,:])
    else:
        def retrieve_data(index):
            #If the data is too large for RAM, this can be swapped around to read from disk
            data = tsd._h5t["data/timeseries/matrix"][index, :]
            return data/sum(data)

    print("Initializing BloomDistance structure")
    bd = BloomDistance(n_objects, distance_function, retrieve_data, 
                       dist_min = 0.001, dist_max=0.15, dist_step=0.005)
    print("Pre-computing distances")
    #This should be set to 1 unless you're using DDTW, but I think that crashes anyways.
    #Worth a shot somewhere with more RAM.
    bd.compute_distances(n_threads=1)
    return bd

In [49]:
#Tunable parameters

nsamples = 180
nclust = 50
nts_per_clust = 10
#noise to signal ratio is 1:1
nsr = 2
shift_amount = 10
signal_variance = 1.75
#Options are "sts","dtw", or "ddtw"
distance_measure = "sts"

In [50]:
param_scores = {}

#for nclust in [1,5,25,50,100,250]:
for i in range(10):
    param = i
    timepoints = np.array(np.cumsum([random.randint(1,15) for i in range(nsamples)]))
    
    def compute_ddtw_distance(data1, data2):
        distance, path = DDTW(data1, data2)
        distance = distance[-1, -1]
        return distance

    def compute_dtw_distance(data1, data2):
        distance, path = fastdtw(data1, data2)
        return distance

    #If we precompute this here, it saves doing it a few million more times
    time_delta = timepoints[1:] - timepoints[0:-1]
    def compute_sts_distance(data1, data2):
        data1_delta = np.array(data1[1:]) - np.array(data1[0:-1])
        data2_delta = np.array(data2[1:]) - np.array(data2[0:-1])
        data1_slope = data1_delta / time_delta
        data2_slope = data2_delta / time_delta
        distance = data1_slope - data2_slope
        distance = np.square(distance)
        distance = np.sqrt(sum(distance))
        return distance

    if distance_measure == "sts":
        distance_function = compute_sts_distance
    elif distance_measure == "dtw":
        distance_function = compute_dtw_distance
    elif distance_measure == "ddtw":
        distance_function = compute_ddtw_distance

    true_signal = generate_data(timepoints, nclust, nts_per_clust, nsr, shift_amount)
    dists = compute_distances(distance_function)

    #Load once so we don't load it a billion times
    tsd = TimeSeriesData("simulation.h5")
    data_matrix = np.empty(tsd._h5t["data/timeseries/matrix"].shape)
    tsd._h5t["data/timeseries/matrix"].read_direct(data_matrix)

    def find_nearest_timeseries(query_data, data_matrix, distance_function, n_threads = 1, n_chunks = 100):
        query_data = query_data/sum(query_data)
        min_distance = 999
        min_index = None
        p_distance_function = partial(distance_function, data2=query_data/sum(query_data))
        # Break the data_matrix into chunks in case the source is on disk
        def chunks(N, nb):
            step = N / nb
            return [(round(step*i), round(step*(i+1))) for i in range(nb)]
        for i, j in chunks(data_matrix.shape[0], n_chunks):
            data = data_matrix[i:j, :]
            for k in range(i, j):
                row = data[k-i, :]
            distance = distance_function(query_data, row/sum(row))
            if distance < min_distance:
                min_distance = distance
                min_index = k
        return min_index

    def score(true_signal, dists, data_matrix, nclust, nts_per_clust, nsr, n_threads = 1):
        nnoise = int(nsr * nclust * nts_per_clust)
        scores = {}
        nearest_neighbour = {}
        print("Finding the nearest neighbours of each signal")
        for i in range(true_signal.shape[0]):
            nearest_index = find_nearest_timeseries(true_signal[i,:]/sum(true_signal[i,:]), 
                                                    data_matrix, distance_function, n_threads)
            nearest_neighbour[i] = nearest_index
        print("Computing scores")
        for distance in dists.dist_range:
            clusters = dists.DBSCAN(distance)
            for i in range(true_signal.shape[0]):
                minimum = i-i%nts_per_clust
                maximum = minimum + nts_per_clust
                ground_truth = np.ones(nclust*nts_per_clust + nnoise)
                ground_truth[minimum:maximum] = 0
                prediction = np.ones(nclust*nts_per_clust + nnoise)
                for cluster in clusters.values():
                    if i in cluster:
                        nearest_cluster = cluster
                        break
                prediction[nearest_cluster] = 0
                score = adjusted_rand_score(ground_truth, prediction)
                if minimum not in scores:
                    scores[minimum] = [score]
                else:
                    scores[minimum].append(score)
        return scores


    scores = score(true_signal, dists, data_matrix, nclust, nts_per_clust, nsr)
    param_scores[param] = scores

Creating required data sets in new HDF5 file at simulation.h5
Loading HDF5 file
Loading data matrix into memory
Initializing BloomDistance structure
After 3000 samples of the distances, the max distance was 0.286982
Pre-computing distances
100.00%
Done!
Finding the nearest neighbours of each signal
Computing scores
Creating required data sets in new HDF5 file at simulation.h5
Loading HDF5 file
Loading data matrix into memory
Initializing BloomDistance structure
After 3000 samples of the distances, the max distance was 0.848667
Pre-computing distances
Bloom filter '0.146' hit capacity, closing
Bloom filter '0.141' hit capacity, closing
Bloom filter '0.136' hit capacity, closing
Bloom filter '0.131' hit capacity, closing
Bloom filter '0.126' hit capacity, closing
Bloom filter '0.121' hit capacity, closing
Bloom filter '0.116' hit capacity, closing
Bloom filter '0.111' hit capacity, closing
Bloom filter '0.106' hit capacity, closing
Bloom filter '0.101' hit capacity, closing
Bloom filter 

These scores represent the best clustering result achievable, across all epsilon values, for that given seed signal. More intuitively, this represents the ability to recover the complete set of signals that are sampled/observed from some underlying process, given knowledge of that process.

In [51]:
n = 0
total_best = []
for param, clusters in param_scores.items():
    n+=1
    best = []
    for cluster_id, scores in clusters.items():
        best.append(max(scores))
    print(str(param) + ": " + str(np.mean(best)))
    total_best.append(np.mean(best))
print("Mean across replicates: %f\nStd. dev. across replicates: %f" % (np.mean(total_best), np.std(total_best)))

0: 0.41407416101030137
1: 0.43006847911389684
2: 0.41256350970337835
3: 0.39982869353282574
4: 0.39866021852533867
5: 0.4224989603062482
6: 0.41400331435225274
7: 0.4022853386310167
8: 0.42283090398663686
9: 0.4006370231360688
Mean across replicates: 0.411745
Std. dev. across replicates: 0.010543


In [ ]:
#Print out the cluster that corresponds to a given tru

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import Scatter, Figure, Layout
init_notebook_mode(connected=True)

def plot_cluster(true_signal, seed_index, dists):
    #Plot some of the clusters
    data = []
    signal = true_signal[seed_index,:]
    observed = tsd._h5t["data/timeseries/matrix"][seed_index,:]
    nearest_index = find_nearest_timeseries(signal/sum(signal), 
                                            data_matrix, distance_function, n_threads=1)
    epsilon = None
    cluster_id = None
    for epsilon in dists.dist_range:
        data = [{'name':'signal', 'x': timepoints, 'y': signal/sum(signal)},
                {'name':'actual', 'x': timepoints, 'y': observed/sum(observed)}]
        cluster_member_indexes = dists.DBSCAN(epsilon, expand_around=nearest_index)
        cluster_id = list(cluster_member_indexes.keys())[0]
        for ts_id in cluster_member_indexes[cluster_id]:
            ts = tsd._h5t["data/timeseries/matrix"][ts_id,:]
            data.append({'name':ts_id, 'y': ts/sum(ts), 'x': timepoints})
        iplot(data)
plot_cluster(true_signal, 2, dists)